In [222]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

In [223]:
import matplotlib.pyplot as plt
import numpy as np
import torch

In [224]:
import bnn.functions
import bnn.network
import bnn.layer
import bnn.loss
import bnn.optimizer

In [225]:
dims = [1, 1]
TBNN = bnn.network.TernBinNetwork(*dims)

In [226]:
TBNN._initialise(W_mean = 0, W_zero_prob=1)

In [227]:
input = torch.ones(1, dtype=torch.int)
desired = -torch.ones(1, dtype=torch.int)

In [228]:
output = TBNN.forward(input)
output

tensor([1], dtype=torch.int32)

In [229]:
loss_func: bnn.loss.LossFunction = bnn.loss.l1

In [230]:
loss = loss_func.forward(output=output, target=desired)
loss

tensor(2)

In [231]:
grad = loss_func.backward(output=output, target=desired)
grad

tensor([-1], dtype=torch.int32)

In [232]:
out_grad = TBNN.backward(grad)
out_grad

tensor([0], dtype=torch.int32)

In [233]:
BernOpt = bnn.optimizer.ExpectationSGD(TBNN.parameters(), lr=1)

In [234]:
TBNN.forward(input)

tensor([1], dtype=torch.int32)

In [237]:
BernOpt.step(number_of_samples=1)

In [236]:
TBNN.forward(input)

tensor([1], dtype=torch.int32)